# Data Glacier Internship

Deliverable - Week 9

Nahari Terena - LISUM15

## Problem Statement

One of the challenge for all Pharmaceutical companies is to understand the persistency of drug as per the physician prescription. To solve this problem ABC pharma company approached an analytics company to automate this process of identification.

### Import Libraries

In [1]:
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

plt.style.use('fivethirtyeight')
warnings.filterwarnings("ignore")

### Import dataset

In [2]:
df = pd.read_csv('Healthcare_dataset.csv', sep = ";")
df.tail()

,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
3419,P3420,Persistent,Female,Caucasian,Not Hispanic,South,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,Y,N,N,N,N,N,N,1
3420,P3421,Persistent,Female,Caucasian,Not Hispanic,South,>75,Unknown,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
3421,P3422,Persistent,Female,Caucasian,Not Hispanic,South,>75,ENDOCRINOLOGY,Specialist,Endo/Onc/Uro,...,N,N,Y,N,N,N,N,N,N,1
3422,P3423,Non-Persistent,Female,Caucasian,Not Hispanic,South,55-65,Unknown,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
3423,P3424,Non-Persistent,Female,Caucasian,Not Hispanic,South,65-75,Unknown,Others,OB/GYN/Others/PCP/Unknown,...,N,N,Y,N,N,N,N,N,N,1


In [3]:
df.shape

(3424, 69)

In [4]:
## Replacing unknown as NA
df_nan = df
df_nan.replace('Unknown', np.nan, inplace = True)

In [5]:
df_nan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424 entries, 0 to 3423
Data columns (total 69 columns):
 #   Column                                                              Non-Null Count  Dtype 
---  ------                                                              --------------  ----- 
 0   Ptid                                                                3424 non-null   object
 1   Persistency_Flag                                                    3424 non-null   object
 2   Gender                                                              3424 non-null   object
 3   Race                                                                3424 non-null   object
 4   Ethnicity                                                           3333 non-null   object
 5   Region                                                              3424 non-null   object
 6   Age_Bucket                                                          3424 non-null   object
 7   Ntm_Speciality          

In [6]:
## Checking if there is any duplicated row
df_nan.duplicated().sum()

0

In [7]:
## Checking kurtosis
df_nan.kurt(axis=0, skipna=True)

Dexa_Freq_During_Rx    74.758378
Count_Of_Risks          0.900486
dtype: float64

In [8]:
## Checking skewness
df_nan.skew(axis=0, skipna=True)

Dexa_Freq_During_Rx    6.808730
Count_Of_Risks         0.879791
dtype: float64

In [9]:
## Summary columns
df_nan[["Dexa_Freq_During_Rx", "Count_Of_Risks"]].describe()

,Dexa_Freq_During_Rx,Count_Of_Risks
count,3424.000000,3424.000000
mean,3.016063,1.239486
std,8.136545,1.094914
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,3.000000,2.000000
max,146.000000,7.000000


In [10]:
#Tukey's method
def tukeys_method(df, variable):
    #Takes two parameters: dataframe & variable of interest as string
    q1 = df[variable].quantile(0.25)
    q3 = df[variable].quantile(0.75)
    iqr = q3-q1
    inner_fence = 1.5*iqr
    outer_fence = 3*iqr
    
    #inner fence lower and upper end
    inner_fence_le = q1-inner_fence
    inner_fence_ue = q3+inner_fence
    
    #outer fence lower and upper end
    outer_fence_le = q1-outer_fence
    outer_fence_ue = q3+outer_fence
    
    outliers_prob = []
    outliers_poss = []
    for index, x in enumerate(df[variable]):
        if x <= outer_fence_le or x >= outer_fence_ue:
            outliers_prob.append(index)
    for index, x in enumerate(df[variable]):
        if x <= inner_fence_le or x >= inner_fence_ue:
            outliers_poss.append(index)
    return outliers_prob, outliers_poss
        
probable_outliers_tm, possible_outliers_tm = tukeys_method(df_nan, "Dexa_Freq_During_Rx")
print(len(probable_outliers_tm))

272


In [11]:
print(len(possible_outliers_tm))

460


In [12]:
## Delete columns with more than 40% of "Unknown answer"
df_del = df_nan.drop(["Risk_Segment_During_Rx", "Tscore_Bucket_During_Rx",  "Change_T_Score", "Change_Risk_Segment"], axis=1)

In [13]:
## transformation for variables with Y and N
df_del.replace(to_replace={'Y': 1, 'N': 0}, inplace=True)

In [14]:
## For Ntm_Speciality: group rare categories as OTHER
df_del['Ntm_Speciality'] = df_del['Ntm_Speciality'].mask(
    df_del['Ntm_Speciality'].map(df_del['Ntm_Speciality'].value_counts(normalize=True)) < 0.01, 'OTHER')

In [15]:
## Ethnicity: group "Unknown" as "not hispanic"
df_del['Ethnicity'].replace(to_replace = {"Unknown" : "Not Hispanic"}, inplace = True)

In [16]:
## Transformation for variables with ">-2.5" and "<=-2.5"
df_del.replace(to_replace={'>-2.5': 1, '<=-2.5': 0}, inplace=True)

In [17]:
## Transformation for variables with "VLR_LR" and "HR_VHR"
df_del.replace(to_replace={'VLR_LR': 1, 'HR_VHR': 0}, inplace=True)

In [18]:
## Replacing the missing values into actual null values. "Unknown" into "NULL"
df_del.replace(["Other/Unknown", "Unknown"], np.nan)

## Replacing all the null values with the mode of the column 
for column in df_del.columns:
    df_del[column].fillna(df_del[column].mode()[0], inplace=True)

In [19]:
## transforming the Age_Bucket variable to numeric. 
df_del.replace(to_replace={
        ">75": 0,
        "65-75": 1,
        "55-65": 2,
        "<55": 3
    },
        inplace=True)

In [20]:
## Splitting the descriptive variables from the target variable
features = df_del.iloc[:, 2:]
target = df_del.Persistency_Flag

In [21]:
features.head()

,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,Gluco_Record_Prior_Ntm,Gluco_Record_During_Rx,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
0,Male,Caucasian,Not Hispanic,West,0,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Male,Asian,Not Hispanic,West,2,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Female,Other/Unknown,Hispanic,Midwest,1,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,0,0,...,0,1,0,0,0,0,0,0,0,2
3,Female,Caucasian,Not Hispanic,Midwest,0,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,0,1,...,0,0,0,0,0,0,0,0,0,1
4,Female,Caucasian,Not Hispanic,Midwest,0,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,1,1,...,0,0,0,0,0,0,0,0,0,1


In [22]:
## Transformations 
imputer = SimpleImputer(strategy="most_frequent")
label_encoder = LabelEncoder()
ohe = OneHotEncoder()

## Fitting transformations 
imputer.fit(features)
label_encoder.fit(target)
ohe.fit(features)

## Transform 
features = imputer.fit_transform(features)
features = ohe.fit_transform(features).toarray()
target = label_encoder.fit_transform(target)

In [23]:
## Assigning the variables X and Y
X = features
Y = target

X, Y

(array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([1, 0, 0, ..., 1, 0, 0]))

In [26]:
df_del.shape

(3424, 65)

In [27]:
## Output clean data
df_del.to_csv("Healthcare_clean", index=False)

In [29]:
df_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424 entries, 0 to 3423
Data columns (total 65 columns):
 #   Column                                                              Non-Null Count  Dtype 
---  ------                                                              --------------  ----- 
 0   Ptid                                                                3424 non-null   object
 1   Persistency_Flag                                                    3424 non-null   object
 2   Gender                                                              3424 non-null   object
 3   Race                                                                3424 non-null   object
 4   Ethnicity                                                           3424 non-null   object
 5   Region                                                              3424 non-null   object
 6   Age_Bucket                                                          3424 non-null   int64 
 7   Ntm_Speciality          